In [27]:
import os 
import json
import pandas as pd
from dotenv import load_dotenv
from qdrant_client import QdrantClient, AsyncQdrantClient
from qdrant_client.http import models
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import  StorageContext, load_index_from_storage, Settings, VectorStoreIndex, Document
from llama_index.core.vector_stores import MetadataFilter, MetadataFilters, FilterOperator, FilterCondition
from llama_index.core.response.notebook_utils import display_source_node


import nest_asyncio

nest_asyncio.apply()

load_dotenv(r"C:\Users\leduc\OneDrive\Desktop\NLP\grab-capstone-project\NavigaTech\AI_modules\.env")

True

In [11]:

embbeding_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")


Settings.embed_model = embbeding_model

c:\Python311\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
qdrant_client = QdrantClient(
    url= os.environ["QDRANT_URL"],
    api_key=os.environ["QDRANT_API_TOKEN"],
)

async_qdrant_client = AsyncQdrantClient(
    url= os.environ["QDRANT_URL"],
    api_key=os.environ["QDRANT_API_TOKEN"],
)

In [3]:
qdrant_client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='job_description')])

In [ ]:
# collection_config = models.VectorParams(
#     size = 1024,
#     distance = models.Distance.COSINE
# )

# qdrant_client.create_collection(
#     collection_name= "job_description",
#     vectors_config= collection_config
# )

True

In [4]:
vector_store = QdrantVectorStore(client=qdrant_client, aclient = async_qdrant_client ,collection_name="job_description")

In [5]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [9]:
with open(r"C:\Users\leduc\OneDrive\Desktop\NLP\grab-capstone-project\NavigaTech\AI_modules\data\rag_data\sample_data.json") as f:
    data = json.load(f)

df = pd.DataFrame(data)
df = df[0:100]

In [12]:
embedding_model = HuggingFaceEmbedding("BAAI/bge-m3")
Settings.embed_model = embedding_model

c:\Python311\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [15]:
documents = []
for index, row in df.iterrows():
    meta = {
        "id": row["id"],
        "title": row["title"],
        "company": row["company"],
        "link": row["link"],
        "keyword": row["search_keyword"],
        "location": row["location"]
    }
    doc = Document(text = row["description"], metadata=meta)
    documents.append(doc)

In [21]:
# index = VectorStoreIndex.from_documents(
#     documents,
#     storage_context=storage_context,
#     show_progress = True
# )
index = VectorStoreIndex.from_vector_store(
    vector_store,
    use_async = True
)

In [28]:
def filter_constructor(fields):
    filters = []
    for k, v in fields.items():
        filters.append(MetadataFilter(key = k, value=v))
    
    final_filters = MetadataFilters(
        filters=filters,
        condition=FilterCondition.AND
    )
    return final_filters

In [33]:
filters = filter_constructor({"location": "United States"})
retriever = index.as_retriever(similarity_top_k=5, filters=filters ,use_async=True)

In [35]:
retriever.retrieve("finance, marketing")

[NodeWithScore(node=TextNode(id_='e4a9dfda-828e-467d-b8d2-37ada4fb37f2', embedding=None, metadata={'id': '4208145067', 'title': 'Rising Star', 'company': 'Panasonic Audio Video Solutions', 'link': 'https://www.linkedin.com/jobs/view/4208145067/', 'keyword': 'Artificial Intelligence', 'location': 'United States'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='de73e15c-a6a8-4265-a9bd-8a9d87f2938d', node_type='4', metadata={'id': '4208145067', 'title': 'Rising Star', 'company': 'Panasonic Audio Video Solutions', 'link': 'https://www.linkedin.com/jobs/view/4208145067/', 'keyword': 'Artificial Intelligence', 'location': 'United States'}, hash='175053591debb70e11c1b222a689ab24ab1fbcf83e6b6971779c85a9f54e755a')}, metadata_template='{key}: {value}', metadata_separator='\n', text="JOB_DESCRIPTION.SHARE.HTMLCAROUSEL_PARAGRAPHJOB_DESCRIPTION.SHARE.HTMLUnited StatesGeneralFull-TimeHybrid / RemotePCONA Panason

In [ ]:
qdrant_client.query_points